In [763]:
import json
import numpy as np
import matplotlib.pyplot as plt
import time
import copy

debug_level = 0
float_formatter = "{:.2f}".format
np.set_printoptions(formatter={'float_kind':float_formatter})

# Aggregation of all line solutions, represented as statistical distribution 
# 1.0 - always set,  -1.0 - always unset 
class Variants:
    def __init__(self, variants):
        self.aggregate = None
        self.count = 0
        for var in variants:
            if self.aggregate is None:
                self.aggregate = np.zeros((var.sum()))
            self.aggregate += var.unpack()
            self.count += 1
        if self.count:
            self.aggregate /= self.count
    def __repr__(self):
        return f"Variants {self.aggregate}"
        
# Packed representation of one line solution
# Even colums represent unset field, odd colums represent set fields
# [0, 1, 3, 2] represents [.X...XX]
class Variant:
    unpacks_done = 0
    def __init__(self, list):
        self.list = list
        self.unpacked = None
    def __repr__(self):
        return f"Variant {self.str()} {[x for x in self.list]}"
    def str(self):
        out = ""
        for i in self.unpack():
            out += 'X' if i == 1 else '.'
        return out
    def sum(self):
        return sum(self.list)
    # Unpack to array where 1: occupied, 0: empty
    def unpack(self):
        if self.unpacked is None:
            self.unpacked = np.zeros(sum(self.list))
            index = 0
            for i in range(len(self.list)):
                for j in range(self.list[i]):
                    self.unpacked[index] = 1 if i%2 else -1
                    index += 1
            Variant.unpacks_done += 1
        return self.unpacked
    
    # Is compatible with already laid down pieces?
    def can_fit(self, known_pieces):
        if not len(known_pieces): return True
        unpacked = self.unpack()
        for i,j in zip(unpacked, known_pieces):
            if (j+i==0): return False   # unpacked can be only 1 or -1. Zero can happen if the other is opposite
        return True
    
        
# Given specification of one line (row or column). E.g. [1,5,2]
class Seq:
    def __init__(self, list):
        self.list = list
        self.allvariants = None
        self.known_pieces = None
    def __repr__(self):
        return f"Seq {self.list}"
    # partial - partially built list of this combination
    # Note that valid_variants will actually remove the already illegal variants from the cached pool
    def variants(self, width, known_pieces=[], partial=[], remove_invalid=False):  
        if self.known_pieces is not None:
            for k1, k2 in zip (self.known_pieces, known_pieces):
                if k1 != 0 and k1 != k2:
                    raise Exception("Can't iterate with less known pieces than before, variants were already removed")
        if self.allvariants is None:
            self.allvariants = [variant for variant in self.gather_variants(width, known_pieces, partial) if variant.can_fit(known_pieces)]

        goodvariants = []
        for variant in self.allvariants:
            if variant.can_fit(known_pieces): 
                yield variant 
                if remove_invalid:
                    goodvariants.append(variant)
        self.known_pieces = np.copy(known_pieces)
        if remove_invalid:
            self.allvariants = goodvariants
            
        
    def gather_variants(self, width, known_pieces, partial=[]):  
        if debug_level > 2: print(f'remain={self.list}, partial={partial}')
        if not Variant(partial).can_fit(known_pieces):
            return
        if not self.list:
            # Successfully emptied all requirements
            finished = Variant(partial + [width - sum(partial)])
            if finished.can_fit(known_pieces):
                yield finished
            return
        # Item that will be placed
        first = self.list[0]
        # Minimum space needed to fit all the remaining ones
        reserved = sum(self.list[1:]) + len(self.list[1:])
        for i in range(0 if not partial else 1, 1 + width - (first + reserved + sum(partial))):
            yield from Seq(self.list[1:]).gather_variants(width, known_pieces, partial + [i, first])
    
    def variant_count(self, width):
        space, needles = (width - (len(self.list)-2 + sum(self.list)), len(self.list))
        if space < 1: return 0
        # print(f'{space=},{needles=}')
        return int(np.math.factorial(space + needles - 1) / (np.math.factorial(space-1) * np.math.factorial(needles)))
    
    # Variant count discounted by the number of known pieces - %SALE %SALE %SALE!
    def variant_discount(self, width, known_pieces):
        if hasattr(known_pieces, '__iter__'): 
            known_pieces = np.count_nonzero(known_pieces)
        return self.variant_count(width) * 0.3**known_pieces # Definitely not exact. The exact distribution matters here.
         # But we don't need exact, just useful and fast enough to choose the next cheap step!

        

# Solving matrix
class Field:
    def __init__(self, rows, cols):
        self.rows = rows
        self.cols = cols
        self.mat = np.zeros((len(self.rows), len(self.cols)))
        self.probs = self.mat.copy() # Probabilities
        self.npass = 0
        self.complexity = 0
        self.more_solutions = False
    def __repr__(self):
        return f'{self.mat}'
    def solve_step(self):
        self.complexity = 0
        while True:
            previous_mat = self.mat.copy()
            row_val, col_val = 1, 0

            # Find the least complex line - sort by complexity. It is only an approximation as also the shape, not just the count of 
            #  known pieces matters here. We're ignoring the shape for simplicity for now.
            lines = [(i,line,row_val,self.mat[i,:]) for i,line in enumerate(self.rows)] + \
                    [(i,line,col_val,self.mat[:,i]) for i,line in enumerate(self.cols)]
            lines = sorted(lines, key = lambda t: t[1].variant_discount(self.mat.shape[t[2]], np.count_nonzero(t[3]))) 

            changed = None       
            for i, line, is_row, known_pieces in lines:
                if changed:
                    break # Last line did add some new information, restart from simplest ones
                if not 0 in known_pieces or np.array_equal(line.known_pieces, known_pieces):
                    continue # Already solved or same as before

                length = self.mat.shape[is_row]
                variants = Variants(line.variants(length, known_pieces, not self.more_solutions))
                self.complexity += variants.count
                if is_row: self.probs[i,:] = variants.aggregate 
                else: self.probs[:,i] = variants.aggregate
                # print(f'Choosing {line=} with complexity:{line.variant_discount(length, known_pieces)}, {variants=}')
                # print(['{:.0f}'.format(x) for x in known_pieces])

                if variants.aggregate is None:
                    return # There is no solution
                for j in range(len(variants.aggregate)):
                    prob = variants.aggregate[j]
                    if prob == -1 or prob == 1:
                        row = j if not is_row else i
                        col = i if not is_row else j
                        if (self.mat[row,col] != prob):
                            changed = ((row, col), (self.mat[row,col], prob))
                            self.mat[row,col] = prob

                self.npass += 1
            if debug_level > 1: print(self.mat)
            changed = (previous_mat != self.mat).any()
            
            # Handle possibility we need to take a guess to continue
            if not changed and not self.is_solved():
                self.more_solutions = True
                guess = copy.copy(self)
                option1, option2 = self.take_a_guess()
                guess.mat = option1
                for advanced in guess.solve_step():
                    self.mat = guess.mat
                    if advanced:
                        yield advanced
                self.complexity += guess.complexity    
                if not self.is_solved():
                    # Guess did not work out, continue with the other option (as long there's only 2)
                    self.mat = option2
                    changed = True
                    
            if not changed:
                break
            yield True
    
    def take_a_guess(self):
        for i in range(self.mat.shape[0]):
            for j in range(self.mat.shape[1]):
                if self.mat[i,j] == 0:
                    option1 = self.mat.copy()
                    option1[i,j] = 1
                    option2 = self.mat.copy()
                    option2[i,j] = 1
                    return (option1, option2)
    
    def is_solved(self):
        return not 0 in field.mat
    
    def last_complexity(self):
        return self.complexity
    
    def visualize(self):
        fig, ax = plt.subplots(figsize=(2, 8))
        ax.pcolormesh(self.mat, cmap='terrain_r')
        ax.set_aspect('equal')
        ax.set_xlim(0, self.mat.shape[1])
        ax.set_ylim(self.mat.shape[0], 0)
        plt.show()
        
    def visualize_probs(self):
        fig, ax = plt.subplots(figsize=(2, 8))
        ax.pcolormesh(self.probs, cmap='terrain_r')
        ax.set_aspect('equal')
        ax.set_xlim(0, self.probs.shape[1])
        ax.set_ylim(self.probs.shape[0], 0)
        plt.show()

In [764]:
debug_level = 1
sequence = [1,4,3,1]
vars = Variants(Seq(sequence).variants(30)) # All possibilities
print(f'Variants: {vars}, count: {vars.count}')
vars = Variants(Seq(sequence).variants(30, [0,0,0,0,0,0,0,0,0,1,0])) # All possibilities
print(f'Variants: {vars}, count: {vars.count}')

# print(Variants(Seq([2,1,1]).variants(7, [0, 0, 0, 1, -1, 0, 0 ]))) # Filtered by known pieces
# print(Variants(Seq([2,1,1]).variants(7, [0, 0, 0, 1, -1, -1, -1 ]))) # Filtered by known pieces
# print(Variants(Seq([2,2,2,2,2,5,5,2]).variants(50)))
print(Seq([2,2,2,2,2,5,5,2]).variant_count(50))
[variant for variant in Seq([1,1,1]).variants(7)]

elems = [3, 1, 2, 1]
width = 12
print(len([print(variant) for variant in Seq(elems).variants(width)]))
print(f'VarCount: {Seq(elems).variant_count(width)}')

Variants: Variants [-0.64 -0.69 -0.68 -0.63 -0.54 -0.43 -0.34 -0.28 -0.24 -0.21 -0.19 -0.18
 -0.17 -0.17 -0.18 -0.20 -0.22 -0.24 -0.27 -0.30 -0.33 -0.36 -0.40 -0.43
 -0.48 -0.54 -0.63 -0.68 -0.69 -0.64], count: 7315
Variants: Variants [-0.65 -0.69 -0.68 -0.64 -0.61 -0.69 -0.38 -0.05 0.34 1.00 0.42 0.04 -0.34
 -0.58 -0.39 -0.32 -0.31 -0.36 -0.39 -0.43 -0.47 -0.50 -0.54 -0.58 -0.62
 -0.67 -0.71 -0.74 -0.74 -0.72], count: 2886
4292145
Variant XXX.X.XX.X.. [0, 3, 1, 1, 1, 2, 1, 1, 2]
Variant XXX.X.XX..X. [0, 3, 1, 1, 1, 2, 2, 1, 1]
Variant XXX.X.XX...X [0, 3, 1, 1, 1, 2, 3, 1, 0]
Variant XXX.X..XX.X. [0, 3, 1, 1, 2, 2, 1, 1, 1]
Variant XXX.X..XX..X [0, 3, 1, 1, 2, 2, 2, 1, 0]
Variant XXX.X...XX.X [0, 3, 1, 1, 3, 2, 1, 1, 0]
Variant XXX..X.XX.X. [0, 3, 2, 1, 1, 2, 1, 1, 1]
Variant XXX..X.XX..X [0, 3, 2, 1, 1, 2, 2, 1, 0]
Variant XXX..X..XX.X [0, 3, 2, 1, 2, 2, 1, 1, 0]
Variant XXX...X.XX.X [0, 3, 3, 1, 1, 2, 1, 1, 0]
Variant .XXX.X.XX.X. [1, 3, 1, 1, 1, 2, 1, 1, 1]
Variant .XXX.X.XX..X [1, 

In [765]:
import requests
import re

def get_actual_puzzle(id):
    url = f'https://www.griddlers.net/en_US/nonogram/-/g/t1679057429974/i01?p_p_lifecycle=2&p_p_resource_id=griddlerPuzzle&p_p_cacheability=cacheLevelPage&_gpuzzles_WAR_puzzles_id={id}&_gpuzzles_WAR_puzzles_lite=false'
    r = requests.get(url)
    match = re.search("var puzzle =(.+)\n\nvar solution", r.text, re.DOTALL)
    js_object = match.group(1)
    # replace single quotes with double quotes
    js_object = js_object.replace("'", '"')
    # wrap keys with double quotes
    js_object = re.sub(r"(\w+)\s?:", r'"\1":', js_object)
    # wrap values with double quotes except for numbers or booleans
    js_object = re.sub(r":\s?(?!(\d+|true|false))(\w+)", r':"\2"', js_object)
    # eradicate continuation , on last element in list   <--  hacky, works only on this one ^_^
    js_object = re.sub(r'",\s*\n', r'"\n', js_object)
    barely_json = json.loads(js_object)
    
    # print(barely_json)
    rows = [[row[1] for row in rowses] for rowses in barely_json["leftHeader"]]
    cols = [[col[1] for col in colses] for colses in barely_json["topHeader"]]
    print(f'Downloaded nonogram {id} from the internets with {len(rows)} rows, {len(cols)} cols')
    return rows, cols

def get_test_puzzle(file):
    with open(file) as f:
        data = json.load(f)
    rows = data['rows']
    cols = data['columns']
    return rows, cols

In [766]:
# Load our next puzzle
# rows, cols = get_test_puzzle('gridler.multisolve.json.txt')
rows, cols = get_actual_puzzle(266162)

srows = [Seq(row) for row in rows]
scols = [Seq(col) for col in cols]
field = Field(srows, scols)

prev = start
step = 0
debug_level = 0
Variant.unpacks_done = 0
start = time.perf_counter()

# Fire the solver
for advanced in field.solve_step():
    step += 1
    if step % 10 == 1:
        print(f'step {step}, time taken so far: {time.perf_counter() - start:.3f} s, complexity: {field.last_complexity()}')
        field.visualize()
    prev = time.perf_counter()
    if not advanced:
        break

if not field.is_solved():
    print("No possible solutions :-(  (or just can't solve them :)")
else:
    print(f"Solved in {step} steps :-)") 
    if field.more_solutions: print(f"There are multiple possible solutions and this is just one of them")
print(f" took {time.perf_counter() - start:.3f} s")
print(f"{Variant.unpacks_done=} (the expensive stuff)")
field.visualize()

# Current state:
#  Still No branching support 


Downloaded nonogram 266162 from the internets with 50 rows, 35 cols


TypeError: 'bool' object is not iterable